In [1]:
import numpy as np
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# LogicGate Class

class LogicGate:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, gate_name, xdata, tdata, input_nodes, hidden_1_nodes, output_nodes, learning_rate, iteration_count):
        
        self.name = gate_name
        
        # 입력 데이터, 정답 데이터 초기화
        self.xdata = xdata.reshape(4, 2)
        self.tdata = tdata.reshape(4, 1)
        
        # 2층 hidden layer unit : 8 개 가정
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_1_nodes)  
        self.b2 = np.random.rand(hidden_1_nodes)
        
        # 3층 output layer unit : 1 개 
        self.W3 = np.random.rand(hidden_1_nodes, output_nodes)
        self.b3 = np.random.rand(output_nodes)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
        
        # 반복횟수 초기화
        self.iteration_count = iteration_count
    
        print(self.name + " object is created")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    # accuracy method 1
    def accuracy1(self, test_xdata, test_tdata):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_xdata)):
            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(test_xdata)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
    
    # accuracy method 2
    def accuracy2(self, test_data):
        
        matched_list = []
        not_matched_list = []
        
        input_data = test_data[ :, 0:-1]
        target_data = test_data[ :, -1]
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.feed_forward()
        
        print("Initial loss value = ", self.loss_val())
        
        start_time = datetime.now()
        
        for step in  range(self.iteration_count):
            
            self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
            self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
            self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
            self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
    
            if (step % 500 == 0):
                print("step = ", step, "loss value = ", self.loss_val())
                
        end_time = datetime.now()
        
        print("")
        print("Elapsed Time => ", end_time - start_time)

In [4]:
and_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
and_tdata = np.array([0, 0, 0, 1])

input_nodes = 2
hidden_1_nodes = 8
output_nodes = 1

lr = 1e-1  # learning rate
count = 10001  # iteration count

# AND Gate 객체 생성 및 training
and_obj = LogicGate("AND", and_xdata, and_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

and_obj.train()

AND object is created
Initial loss value =  10.548530290890318
step =  0 loss value =  6.934486564262593
step =  500 loss value =  0.2861705307139749
step =  1000 loss value =  0.05498348356323624
step =  1500 loss value =  0.026923439839780818
step =  2000 loss value =  0.017179950242187922
step =  2500 loss value =  0.01239664601594849
step =  3000 loss value =  0.009600432700455882
step =  3500 loss value =  0.007783494852327331
step =  4000 loss value =  0.006515977135192804
step =  4500 loss value =  0.0055854687044110595
step =  5000 loss value =  0.004875615900477729
step =  5500 loss value =  0.004317620889315068
step =  6000 loss value =  0.0038683302907835705
step =  6500 loss value =  0.0034993723540316067
step =  7000 loss value =  0.0031913637382771625
step =  7500 loss value =  0.0029306322464944853
step =  8000 loss value =  0.0027072703167002744
step =  8500 loss value =  0.0025139305968771538
step =  9000 loss value =  0.0023450542158958473
step =  9500 loss value =  0

In [5]:
# AND Gate prediction
print(and_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

(true_list, false_list) = and_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = and_obj.accuracy2(test_data)

AND 

Accuracy =>  1.0
Accuracy =>  1.0


In [6]:
or_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
or_tdata = np.array([0, 1, 1, 1])

input_nodes = 2
hidden_1_nodes = 8
output_nodes = 1

lr = 1e-1  # learning rate
count = 10001  # iteration count

# OR Gate 객체 생성 및 training
or_obj = LogicGate("OR", or_xdata, or_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

or_obj.train()

OR object is created
Initial loss value =  3.0462715803567337
step =  0 loss value =  2.779539065517594
step =  500 loss value =  0.09426366535314372
step =  1000 loss value =  0.029217931836749887
step =  1500 loss value =  0.016184515265581647
step =  2000 loss value =  0.010932309972421143
step =  2500 loss value =  0.008157214603789766
step =  3000 loss value =  0.006460598524854317
step =  3500 loss value =  0.0053238838991290826
step =  4000 loss value =  0.004512872913347457
step =  4500 loss value =  0.003907083900483349
step =  5000 loss value =  0.0034384976695042003
step =  5500 loss value =  0.003065934087199561
step =  6000 loss value =  0.0027630648654659536
step =  6500 loss value =  0.0025123040431430676
step =  7000 loss value =  0.002301475338829838
step =  7500 loss value =  0.002121890977007663
step =  8000 loss value =  0.001967192190664654
step =  8500 loss value =  0.0018326217821013735
step =  9000 loss value =  0.0017145523579908584
step =  9500 loss value =  0

In [7]:
# OR Gate prediction
print(or_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

(true_list, false_list) = or_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = or_obj.accuracy2(test_data)

OR 

Accuracy =>  1.0
Accuracy =>  1.0


In [8]:
nand_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
nand_tdata = np.array([1, 1, 1, 0])

input_nodes = 2
hidden_1_nodes = 8
output_nodes = 1

lr = 1e-1  # learning rate
count = 10001  # iteration count

# OR Gate 객체 생성 및 training
nand_obj = LogicGate("NAND", nand_xdata, nand_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

nand_obj.train()

NAND object is created
Initial loss value =  3.7211275929836622
step =  0 loss value =  3.2722266161411544
step =  500 loss value =  0.15834416199857254
step =  1000 loss value =  0.04502988747445658
step =  1500 loss value =  0.024198110400919598
step =  2000 loss value =  0.016095016852262264
step =  2500 loss value =  0.011896312338668714
step =  3000 loss value =  0.009361348115848479
step =  3500 loss value =  0.007677885528210424
step =  4000 loss value =  0.00648468855802015
step =  4500 loss value =  0.0055979982518727925
step =  5000 loss value =  0.004914965381269138
step =  5500 loss value =  0.004373751420566676
step =  6000 loss value =  0.003935043301970196
step =  6500 loss value =  0.003572705934158209
step =  7000 loss value =  0.00326871652568518
step =  7500 loss value =  0.0030102604839609857
step =  8000 loss value =  0.0027879870006544084
step =  8500 loss value =  0.0025949191699733342
step =  9000 loss value =  0.0024257497377165264
step =  9500 loss value =  0.

In [9]:
# NAND Gate prediction
print(nand_obj.name, "\n")

test_data = np.array([ [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

(true_list, false_list) = nand_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = nand_obj.accuracy2(test_data)

NAND 

Accuracy =>  1.0
Accuracy =>  1.0


In [10]:
xor_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
xor_tdata = np.array([0, 1, 1, 0])

input_nodes = 2
hidden_1_nodes = 8
output_nodes = 1

lr = 1e-1  # learning rate
count = 10001  # iteration count

# XOR Gate 객체 생성 및 training
xor_obj = LogicGate("XOR", xor_xdata, xor_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

xor_obj.train()

XOR object is created
Initial loss value =  5.443748485035505
step =  0 loss value =  4.118547079996691
step =  500 loss value =  2.0385613690091304
step =  1000 loss value =  0.32263007076573974
step =  1500 loss value =  0.12769366344631763
step =  2000 loss value =  0.07481041082092182
step =  2500 loss value =  0.05138319479038142
step =  3000 loss value =  0.03845631294367048
step =  3500 loss value =  0.030375529575756233
step =  4000 loss value =  0.024898794061288913
step =  4500 loss value =  0.020969882978593972
step =  5000 loss value =  0.018029769357941955
step =  5500 loss value =  0.015756540239745813
step =  6000 loss value =  0.01395255056275137
step =  6500 loss value =  0.012490168659808475
step =  7000 loss value =  0.01128353845022615
step =  7500 loss value =  0.010272922158525509
step =  8000 loss value =  0.00941555725036755
step =  8500 loss value =  0.008680081581834474
step =  9000 loss value =  0.008043007356881892
step =  9500 loss value =  0.00748641959484

In [11]:
# XOR Gate prediction
print(xor_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

(true_list, false_list) = xor_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = xor_obj.accuracy2(test_data)

XOR 

Accuracy =>  1.0
Accuracy =>  1.0
